In [1]:
using DataFrames
using Recommender: Movielens100k, load_dataset, leave_one_out_split, ItemkNN, fit!, predict, evaluate, RecallAtK

In [ ]:
ml100k = Movielens100k()
download(ml100k)
rating, user, movie = load_dataset(ml100k);

In [ ]:
df = DataFrame(rating)
first(df, 10)

,userid,movieid,rating,timestamp
,Int64,Int64,Int64,Int64
1,196,242,3,881250949
2,186,302,3,891717742
3,22,377,1,878887116
4,244,51,2,880606923
5,166,346,1,886397596
6,298,474,4,884182806
7,115,265,2,881171488
8,253,465,5,891628467
9,305,451,3,886324817


In [ ]:
df[!, :rating] .= 1
first(df, 10)

,userid,movieid,rating,timestamp
,Int64,Int64,Int64,Int64
1,196,242,1,881250949
2,186,302,1,891717742
3,22,377,1,878887116
4,244,51,1,880606923
5,166,346,1,886397596
6,298,474,1,884182806
7,115,265,1,881171488
8,253,465,1,891628467
9,305,451,1,886324817


In [ ]:
df = leave_one_out_split(df, col_group = :userid, col_sort = :timestamp);

In [ ]:
df_train = df[df.data_split.==:train, :]
df_valid = df[df.data_split.==:valid, :]
df_test = df[df.data_split.==:test, :]
first(df_train, 10)

,userid,movieid,rating,timestamp,data_split
,Int64,Int64,Int64,Int64,Symbol
1,196,1118,1,881252128,train
2,196,108,1,881252110,train
3,196,411,1,881252090,train
4,196,580,1,881252056,train
5,196,67,1,881252017,train
6,196,692,1,881252017,train
7,196,25,1,881251955,train
8,196,13,1,881251955,train
9,196,762,1,881251955,train


In [ ]:
recall100 = RecallAtK(100)

RecallAtK(100)

In [ ]:
using TreeParzen

In [ ]:
space = Dict(
    :topk=>HP.QuantUniform(:topk, 10., 500., 1.),
    :shrink=>HP.LogUniform(:shrink, log(1e-3), log(1e3)),
    :weighting=>HP.Choice(:weighting, 
        [
            Dict(:weighting=>:dummy, :weighting_at_inference=>false),
            Dict(:weighting=>:tfidf, :weighting_at_inference=>false),
            Dict(:weighting=>:bm25, :weighting_at_inference=>HP.Choice(:weighting_at_inference, [true, false]))
        ]
    ),
    :normalize=>HP.Choice(:normalize, [true, false])
)

Dict{Symbol, TreeParzen.Types.AbstractDelayed} with 4 entries:
  :weighting => Choice(Param(:weighting, RandIndex(3)), Dict{Symbol, Any}[Dict(…
  :topk      => QuantUniform(:topk, QuantUniform(10.0, 500.0, 1.0))
  :normalize => Choice(Param(:normalize, RandIndex(2)), Bool[1, 0])
  :shrink    => LogUniform(:shrink, LogUniform(-6.90776, 6.90776))

In [ ]:
function invert_output(params)
    k = convert(Int, params[:topk])
    model = ItemkNN(k, params[:shrink],params[:weighting][:weighting],params[:weighting][:weighting_at_inference],params[:normalize])
    result = evaluate(model, df_train, df_valid, recall100, 100, col_user=:userid, col_item=:movieid, col_rating=:rating)
    @show params, result
    return -result
end

invert_output (generic function with 1 method)

In [ ]:
best = fmin(invert_output, space, 100, logging_interval=-1)

params = Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :bm25, :weighting_at_inference => false), :topk => 139.0, :normalize => true, :shrink => 1.831248046147268)
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :bm25, :weighting_at_inference => false), :topk => 139.0, :normalize => true, :shrink => 1.831248046147268), 0.37751855779427357)


┌ Info: fmin: 1 / 1 trials carried out
└ @ TreeParzen.API /Users/keisuke.yanagi/.julia/packages/TreeParzen/Iw2mh/src/API.jl:176
┌ Info: Successfully completed fmin 
└ @ TreeParzen.API /Users/keisuke.yanagi/.julia/packages/TreeParzen/Iw2mh/src/API.jl:231


Dict{Symbol, Any} with 4 entries:
  :weighting => Dict{Symbol, Any}(:weighting=>:bm25, :weighting_at_inference=>f…
  :topk      => 139.0
  :normalize => true
  :shrink    => 1.83125

In [ ]:
best_model = ItemkNN(convert(Int, best[:topk]), best[:shrink],best[:weighting],best[:weighting_at_inference],best[:normalize])
evaluate(best_model, df_train_valid, df_test, recall100, 100, col_user=:userid, col_item=:movieid, col_rating=:rating)

LoadError: KeyError: key :weighting_at_inference not found